# Transforming graph data to pandas dataframe

To make a pandas dataframe from the graphdata it is first necessary to import the data. How to do this is explained on: <br> https://github.com/alkem-io/analytics-playground 

In [2]:
%run Functions.ipynb

To use some of the funtions used in this notebook you first need to download the Functions notebook, which is available in jupyter notebook and spyder.

In [3]:
import pandas as pd
import json
from pandas import json_normalize

When you have downloaded the matrix as JSON-format, which is used to create the graph. With open you can open the data to make it as a pandas dataframe. <br><br>
Within the variable_name['nodes'] it is possible to reach the information of the hubs, challenges, opportunities and contributors

In [4]:
with open('transformed-graph-data-new.json', 'r', encoding='utf-8') as file:
    variable_name = json.load(file)

### Node information

Get all the node information of the hubs, contributors, challenges and opportunities

In [6]:
nodes = variable_name['nodes']

In [26]:
hubs = json_normalize(nodes['hubs'])

In [10]:
contributors = json_normalize(nodes['contributors'])

In [30]:
challenges = json_normalize(nodes['challenges'])

In [34]:
opportunities = json_normalize(nodes['opportunities'])

Save the hub, challenge, opportunity and contributors as csv so you can reach them any time necessary

In [22]:
hubs.to_csv('graph_hubs_2023')
challenges.to_csv('graph_challenges_2023')
opportunities.to_csv('graph_opportunities_2023')
contributors.to_csv('graph_contributors_2023')

In [23]:
print('column available for the hubs, challenges, opportunities and contributors are:')
for i in hubs.columns:
    print(i)

column available for the hubs, challenges, opportunities and contributors are:
id
nameID
displayName
type
group
weight
url
avatar
country
city
lon
lat
leadOrgsCount


In [24]:
print(f'amount of hubs: {len(hubs)}')
print(f'amount of challenges: {len(challenges)}')
print(f'amount of opportunities: {len(opportunities)}')
print(f'amount of contributors: {len(contributors)}')

amount of hubs: 22
amount of challenges: 68
amount of opportunities: 140
amount of contributors: 874


### Edge information

In [25]:
edges = json_normalize(variable_name['edges'])

In [26]:
print('column available for the edges are:')
for i in edges.columns:
    print(i)

column available for the edges are:
sourceID
targetID
source
target
weight
type
group


The sourceID = source, and targetID=target

## Connect sourceID with node names
the goal for creating this dataframe is to make a dataframe which shows in the rows the different users and in the journeys the communities. The value is 1 if a user is a member of the journey. The only infromation that is kept of the hubs, challenges, opportunity and contributors are the id, name, type and group

In [28]:
hubs_min = hubs.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat', 'leadOrgsCount'], axis=1)
challenges_min = challenges.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat', 'leadOrgsCount'], axis=1)
opportunities_min = opportunities.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat', 'leadOrgsCount'], axis=1)
users_min = contributors.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat'], axis=1)

### Connect Hubs

In [29]:
merge = pd.merge(edge_min, hubs_min, left_on='source', right_on='id', how='left')

In [30]:
amount_hubs_source = len(merge[merge['displayName'].notnull()])
print(f'there are {amount_hubs_source} amount of hubs as a source node')

there are 0 amount of hubs as a source node


In [31]:
merge = merge.drop(['id'], axis=1)

In [32]:
merge = merge.rename(columns={'displayName':'name Source', 'type_x':'type Edge', 'type_y':'type Source', 
                              'group_y':'group Source', 'group_x':'group Edge'})

In [33]:
test_amount_hubs_source = len(merge[merge['name Source'].notnull()])
print(f'there should be {amount_hubs_source} amount of rows')
print(f'there are {test_amount_hubs_source} amount of hubs as a source node')


there should be 0 amount of rows
there are 0 amount of hubs as a source node


### Connect Challenges

In [34]:
merge2 = pd.merge(merge, challenges_min, left_on='source', right_on='id', how='left')

In [35]:
amount_challenges_source = len(merge2[merge2['displayName'].notnull()])
print(f'there are {amount_challenges_source} amount of challenges as a source node')

there are 68 amount of challenges as a source node


In [36]:
merge2['name Source'] = merge2['name Source'].combine_first(merge2['displayName'])
merge2['type Source'] = merge2['type Source'].combine_first(merge2['type'])
merge2['group Source'] = merge2['group Source'].combine_first(merge2['group'])

In [37]:
merge2 = merge2.drop(['id', 'displayName', 'type', 'group'], axis=1)

In [38]:
test_amount_challenges_source = len(merge2[merge2['name Source'].notnull()])
print(f'there should be {amount_challenges_source+amount_hubs_source} amount of rows')
print(f'there are {test_amount_challenges_source} amount of hubs and challenges as a source node')


there should be 68 amount of rows
there are 68 amount of hubs and challenges as a source node


### Connect Opportunities

In [39]:
merge3 = pd.merge(merge2, opportunities_min, left_on='source', right_on='id', how='left')

In [40]:
amount_opportunities_source = len(merge3[merge3['displayName'].notnull()])
print(f'there are {amount_opportunities_source} amount of challenges as a source node')

there are 140 amount of challenges as a source node


In [41]:
merge3['name Source'] = merge3['name Source'].combine_first(merge3['displayName'])
merge3['type Source'] = merge3['type Source'].combine_first(merge3['type'])
merge3['group Source'] = merge3['group Source'].combine_first(merge3['group'])

In [42]:
merge3 = merge3.drop(['id', 'displayName', 'type', 'group'], axis=1)

In [43]:
test_amount_opportunities_source = len(merge3[merge3['name Source'].notnull()])
print(f'there should be {amount_opportunities_source}+{+amount_challenges_source}+{amount_hubs_source}={amount_opportunities_source+amount_challenges_source+amount_hubs_source} amount of rows')
print(f'there are {test_amount_opportunities_source} amount of hubs, challenges and opportunities as a source node')

there should be 140+68+0=208 amount of rows
there are 208 amount of hubs, challenges and opportunities as a source node


### Connect Contributors

In [44]:
merge4 = pd.merge(merge3, users_min, left_on='source', right_on='id', how='left')

In [45]:
amount_users_source = len(merge4[merge4['displayName'].notnull()])
print(f'there are {amount_users_source} amount of challenges as a source node')

there are 1363 amount of challenges as a source node


In [46]:
merge4['name Source'] = merge4['name Source'].combine_first(merge4['displayName'])
merge4['type Source'] = merge4['type Source'].combine_first(merge4['type'])
merge4['group Source'] = merge4['group Source'].combine_first(merge4['group'])

In [47]:
merge4 = merge4.drop(['id', 'displayName', 'type', 'group'], axis=1)

In [48]:
test_amount_users_source = len(merge4[merge4['name Source'].notnull()])
print(f'there should be {amount_users_source}+{amount_opportunities_source}+{amount_challenges_source}+{amount_hubs_source}={amount_users_source+amount_opportunities_source+amount_challenges_source+amount_hubs_source} amount of rows')
print(f'there are {test_amount_users_source} amount of hubs, challenges, opportunities and contributors as a source node')

there should be 1363+140+68+0=1571 amount of rows
there are 1571 amount of hubs, challenges, opportunities and contributors as a source node


## Connect targetID with node names

### Hubs

In [49]:
merge5 = pd.merge(merge4, hubs_min, left_on='target', right_on='id', how='left')

In [50]:
amount_hubs_target = len(merge5[merge5['displayName'].notnull()])
print(f'there are {amount_hubs_target} amount of hubs as a target node')

there are 616 amount of hubs as a target node


In [51]:
merge5 = merge5.rename(columns={'displayName':'name Target', 'type':'type Target', 'group':'group Target'})

In [52]:
merge5 = merge5.drop(['id'], axis=1)

In [53]:
test_amount_hubs_target= len(merge5[merge5['name Target'].notnull()])
print(f'there should be {amount_hubs_target} amount of rows')
print(f'there are {test_amount_hubs_target} amount of hubs as a source node')

there should be 616 amount of rows
there are 616 amount of hubs as a source node


### Challenges

In [54]:
merge6 = pd.merge(merge5, challenges_min, left_on='target', right_on='id', how='left')

In [55]:
amount_challenges_target = len(merge6[merge6['displayName'].notnull()])
print(f'there are {amount_challenges_target} amount of hubs as a target node')

there are 547 amount of hubs as a target node


In [56]:
merge6['name Target'] = merge6['name Target'].combine_first(merge6['displayName'])
merge6['type Target'] = merge6['type Target'].combine_first(merge6['type'])
merge6['group Target'] = merge6['group Target'].combine_first(merge6['group'])

In [57]:
merge6 = merge6.drop(['id', 'displayName', 'type', 'group'], axis=1)

In [58]:
test_amount_challenges_target = len(merge6[merge6['name Target'].notnull()])
print(f'there should be {amount_challenges_target}+{amount_hubs_target}={amount_challenges_target+amount_hubs_target} amount of rows')
print(f'there are {test_amount_challenges_target} amount of hubs and challenges as a target node')

there should be 547+616=1163 amount of rows
there are 1163 amount of hubs and challenges as a target node


### Opportunities

In [59]:
merge7 = pd.merge(merge6, opportunities_min, left_on='target', right_on='id', how='left')

In [60]:
amount_opportunities_target = len(merge7[merge7['displayName'].notnull()])
print(f'there are {amount_opportunities_target} amount of challenges as a source node')

there are 408 amount of challenges as a source node


In [61]:
merge7['name Target'] = merge7['name Target'].combine_first(merge7['displayName'])
merge7['type Target'] = merge7['type Target'].combine_first(merge7['type'])
merge7['group Target'] = merge7['group Target'].combine_first(merge7['group'])

In [62]:
merge7 = merge7.drop(['id', 'displayName', 'type', 'group'], axis=1)

In [63]:
test_amount_opportunities_target = len(merge7[merge7['name Target'].notnull()])
print(f'there should be {amount_opportunities_target}+{amount_challenges_target}+{amount_hubs_target}={amount_opportunities_target+amount_challenges_target+amount_hubs_target} amount of rows')
print(f'there are {test_amount_opportunities_target} amount of hubs challenge and opportunities as a target node')

there should be 408+547+616=1571 amount of rows
there are 1571 amount of hubs challenge and opportunities as a target node


### Contributors

In [64]:
merge8 = pd.merge(merge7, users_min, left_on='target', right_on='id', how='left')

In [65]:
amount_users_target = len(merge8[merge8['displayName'].notnull()])
print(f'there are {amount_users_target} amount of contributors as a target node')

there are 0 amount of contributors as a target node


In [66]:
merge8 = merge8.drop(['id', 'displayName', 'type', 'group'], axis=1)

## Exporting dataframe to csv

In [67]:
merge8.to_csv('graphdata_pandas_2023.csv', index=False)

In [68]:
print('column available for the dataframe are:')
for i in merge8.columns:
    print(i)

column available for the dataframe are:
source
target
type Edge
group Edge
name Source
type Source
group Source
name Target
type Target
group Target


# Making predition tabel

In [69]:
data = pd.read_csv('graphdata_pandas_2023.csv')

type of the node goes next to the name, this makes it easier to see what type a node is

In [70]:
for i in range(len(data)):
    data['name Source'][i] = data['name Source'][i] + str(" ") + data['type Source'][i]
    data['name Target'][i] = data['name Target'][i] + str(" ") + data['type Target'][i]

In [71]:
data[data['name Source']==data['name Target']]

,source,target,type Edge,group Edge,name Source,type Source,group Source,name Target,type Target,group Target


There arent any source nodes and target nodes that refer to themselfes

In [72]:
data['type Source'].value_counts()

user            1256
opportunity      140
organization     107
challenge         68
Name: type Source, dtype: int64

In [73]:
data['type Target'].value_counts()

hub            616
challenge      547
opportunity    408
Name: type Target, dtype: int64

This seems like a lot of different journeys and contributors, but every connection a user has is put on a different row, so if a user is a member of four different communities, it is counted as 4 different users.

In [74]:
data

,source,target,type Edge,group Edge,name Source,type Source,group Source,name Target,type Target,group Target
0,01ef561d-2450-4bb5-841a-0c678ea84170,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,member,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,Marianne Schimmel user,user,contributors,Publieke dienstverlening hub,hub,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5
1,029997d8-0a26-4de6-9cb3-356f9175d344,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,member,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,Ben Schuttenbeld user,user,contributors,Publieke dienstverlening hub,hub,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5
2,0a105d73-fce3-4afd-a15d-bbdf109d8317,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,member,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,Wikke Alphenaar user,user,contributors,Publieke dienstverlening hub,hub,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5
3,0bbf579f-4b91-43f7-8f80-27f65f731cdb,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,member,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,Arthur Maring user,user,contributors,Publieke dienstverlening hub,hub,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5
4,14e509c0-96d5-4fa7-8092-f17b1096dbee,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,member,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5,Martin de Bijl user,user,contributors,Publieke dienstverlening hub,hub,6d2fb355-5f0f-4545-b2d1-945f37ecbbb5
...,...,...,...,...,...,...,...,...,...,...
1566,0bc15a44-15d3-4460-9587-7a8f19a98f53,65dc2f1e-9d9b-42e0-b250-5e9f7d0d534b,member,bb9b546f-2703-475e-8410-f010f5fe7eab,Stephan Corporaal user,user,contributors,Monteurs vinden opportunity,opportunity,bb9b546f-2703-475e-8410-f010f5fe7eab
1567,97403d8a-58f9-45f8-884c-e8187c01786b,9fae45ff-ec64-45b5-98a6-d3652c999442,child,bb9b546f-2703-475e-8410-f010f5fe7eab,Partnerschapskaart opportunity,opportunity,bb9b546f-2703-475e-8410-f010f5fe7eab,Werkomgeving voor Learning community ontwikkel...,challenge,bb9b546f-2703-475e-8410-f010f5fe7eab
1568,3f620917-05c7-4e79-8e9b-c151f94e5ae1,97403d8a-58f9-45f8-884c-e8187c01786b,member,bb9b546f-2703-475e-8410-f010f5fe7eab,Serena Scholte user,user,contributors,Partnerschapskaart opportunity,opportunity,bb9b546f-2703-475e-8410-f010f5fe7eab
1569,4fa2594d-a8c5-4369-8530-09dd8ba69a1c,9ad42f02-c5cf-4dfc-aab9-d1530dc19174,child,15264ed3-fb76-4233-aa1b-36811c1f6ad9,Digital Literacy Course opportunity,opportunity,15264ed3-fb76-4233-aa1b-36811c1f6ad9,Tech for Social Good challenge,challenge,15264ed3-fb76-4233-aa1b-36811c1f6ad9


In [75]:
data.to_csv('data_graph_matrix_2023.csv')